#### Potřebné knihovny

---


In [ ]:
import os
import string

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from tensorflow.keras import callbacks
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model

<br>

#### Převedení obrázku na potřebné rozlišení

---

Některé obrázky s captchou nemají potřebné rozlišení 200x50 pixelů. Takové snímky je potřeba upravit.

<br>

Třída `Picture` slouží pouze pro inicializaci existujících obrázků.

<br>

Třída `Modified` dědí z třídy `Picture` její instanční atributy a slouží pro modifikaci stávajících parametrů a vytvoření nového obrázku.

In [ ]:
class Picture:
    def __init__(self, path: str, name: str, width: int, height: int) -> None:
        self.path = path
        self.name = name
        self.width = width
        self.height = height
        
    def initiate_image(self, imread_flag:int) -> np.ndarray:
        return cv2.imread(self.path, imread_flag)
        
    def get_dimension_of_image(self, img: np.ndarray) -> tuple:
        return img.shape
    
    def save_image(self, img: np.ndarray) -> None:
        cv2.imwrite(self.name, img)


class Modifier(Picture):
    def is_resolution_same(self, img: np.ndarray) -> bool:
        original_width, original_height, *number = self.get_dimension_of_image(img)
        return (original_width, original_height) == (self.width, self.height)
    
    def resize_image(self, img: np.ndarray) -> np.ndarray:
        return cv2.resize(
            img,
            (self.height, self.width),
            interpolation=cv2.INTER_NEAREST
        )

In [ ]:
# testing of class instance with example 'nh4ut.png'
testing_png = Modifier("captchas/qft8t.png", "qft8t_resized.png", 50, 200)
image = testing_png.initiate_image(cv2.IMREAD_UNCHANGED)

if testing_png.is_resolution_same(image):
    print(f"Resolution is already correct {testing_png.get_dimension(image)}")
else:
    resized = testing_png.resize_image(image)
    testing_png.save_image(resized)

#### Odstranění šumu a přebytečných čar z obrázku

---

Některé z obrázků mohou obsahovat přidaný šum teček a jiných patvarů, které je nutné zjemnit pro následné vyhodnocování.

In [ ]:
%matplotlib inline

class Erosion(Picture):
    def apply_adaptive_thresholding(self):
        image = self.initiate_image(cv2.IMREAD_GRAYSCALE)
        return cv2.adaptiveThreshold(
            image,
            255,
            cv2.ADAPTIVE_THRESH_MEAN_C,  # ADAPTIVE_THRESH_GAUSSIAN_C
            cv2.THRESH_BINARY,
            199,
            5
        )
        
    def dilate_image(self, threshold, num_of_iteration: int = 1):
        kernel = np.ones((3,1), np.uint8)
#         img_erosion = cv2.erode(threshold, kernel, iterations=num_of_iteration)
        return cv2.dilate(threshold, kernel, iterations=num_of_iteration)

In [ ]:
# testing of class instance with example 'nh4ut.png'
erosion = Erosion("qft8t_resized.png", "qft8t_clean.png", 50, 200)
threshold = erosion.apply_adaptive_thresholding()
result = erosion.dilate_image(threshold)
plt.imshow(result) 
erosion.save_image(result)

#### Predikce hodnot

---

Nachystání modelu (poznámky nejsou kompletní)

In [ ]:
def recognize_captcha():
    run_recognition()

    
def run_recognition():
    symbols = string.ascii_lowercase + string.digits
    number_of_values = len(symbols)
    image_shape = (50, 200, 1)
    model = create_model(image_shape, number_of_values)
    X, y = preprocess_data(symbols, number_of_values)
    
    X_train, y_train = X[:970], y[:, :970]
    X_test, y_test = X[970:], y[:, 970:]
    
#     model.summary()

    hist = model.fit(
        X_train,
        [y_train[0], y_train[1], y_train[2], y_train[3], y_train[4]],
        batch_size=32,
        epochs=30,
        verbose=1, 
        validation_split=0.2
    )

     # evaluate the precission
#     score= model.evaluate(
#         X_test,
#         [y_test[0], y_test[1], y_test[2], y_test[3], y_test[4]],
#         verbose=1
#     )
#     print('Test Loss and accuracy:', score)
    
    # test some captcha samples 
    return model

#     model.evaluate(X_test, [y_test[0], y_test[1], y_test[2], y_test[3], y_test[4]])
#     print(predict(model, symbols, './solver/data/samples/3dgmf.png'))  # 3dgmf
#     print(predict(model, symbols, './solver/data/samples/5gcd3.png'))  # 5gcd3
#     print(predict(model, symbols, './solver/data/samples/xymfn.png'))  # xynfn
#     print(predict(model, symbols, './solver/data/samples/yyn57.png'))  # yyn57
#     print(predict(model, symbols, './solver/data/samples/e43ym.png'))  # e43ym
#     print(predict(model, symbols, '4neuf_clean.png'))  # ?


def create_model(img_shape: tuple, num_of_vals: int):
    """
    Links:
    - https://keras.io/api/layers/core_layers/input/
    - https://keras.io/api/layers/convolution_layers/convolution2d/
    - https://keras.io/api/layers/pooling_layers/max_pooling2d/
    - https://keras.io/api/layers/normalization_layers/batch_normalization/
    
    Description:
    1. Input() is used to instantiate a Keras tensor,
    2. 2D convolution layer (e.g. spatial convolution over images).
    3. Max pooling operation for 2D spatial data.
    4. Batch normalization applies a transformation that maintains
       the mean output close to 0 and the output standard
       deviation close to 1.
    5. Compile model and return it.
    """
    image = layers.Input(shape=img_shape)
    
    mp3 = process_image_through_convulations(image)
    flattened_vectors = get_flattened_vector(mp3, num_of_vals)

    model = Model(image, flattened_vectors)
    model.compile(
        loss='categorical_crossentropy',
        optimizer='adam',
        metrics=["accuracy"]
    )
    
    return model


def process_image_through_convulations(img_shape: tuple):
    convolution_1 = layers.Conv2D(16, (3, 3), padding='same', activation='relu')(img_shape)
    mp1 = layers.MaxPooling2D(padding='same')(convolution_1)
    convolution_2 = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(mp1)
    mp2 = layers.MaxPooling2D(padding='same')(convolution_2)
    convolution_3 = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(mp2)
    b_normalization = layers.BatchNormalization()(convolution_3)
    return layers.MaxPooling2D(padding='same')(b_normalization)
    

def adjust_layer(flat, num_of_vals):
    dens1 = layers.Dense(64, activation='relu')(flat)
    drop = layers.Dropout(0.5)(dens1)
    return layers.Dense(num_of_vals, activation='sigmoid')(drop)

    
def get_flattened_vector(mp, num_of_vals):
    """
    Links:
    - https://keras.io/api/layers/regularization_layers/dropout/
    - https://keras.io/api/layers/core_layers/dense/
    
    Description:
    1. The Dropout layer randomly sets input units to 0 with
       a frequency of rate at each step during training time,
       which helps prevent overfitting. Inputs not set to 0
       are scaled up by 1/(1 - rate) such that the sum over
       all inputs is unchanged.
    """
    flattened = layers.Flatten()(mp)
    return [
        adjust_layer(flattened, num_of_vals)
        for _ in range(5)
    ]


def preprocess_data(symbols: str, symbols_len: int):
    rel_path = "./solver/data/samples"
    number_of_samples = len(os.listdir(rel_path))
    X = np.zeros((number_of_samples, 50, 200, 1))
    y = np.zeros((5, number_of_samples, symbols_len))
    
    for index, picture in enumerate(os.listdir(rel_path)):
        img = cv2.imread(os.path.join(rel_path, picture), cv2.IMREAD_GRAYSCALE)
        captcha = os.path.splitext(picture)[0]
        
        if len(captcha) < 6:
            img = img / 255.0
            img = np.reshape(img, (50, 200, 1))
            
            targets = np.zeros((5, symbols_len))
            
            for nest_index, sign in enumerate(captcha):
                sign_index = symbols.find(sign)
                targets[nest_index, sign_index] = 1
            
            X[index] = img
            y[:, index] = targets
    return X, y


def predict(model, symbols: str, filepath: str) -> str:
    img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    
    if img is not None:
        img = img / 255.0
    else:
        print("Not detected")
        
    res = np.array(model.predict(img[np.newaxis, :, :, np.newaxis]))
    ans = np.reshape(res, (5, 36))
    l_ind = []
    probs = []
    
    for a in ans:
        l_ind.append(np.argmax(a))
        #probs.append(np.max(a))

    capt = ''
    
    for l in l_ind:
        
        capt += symbols[l]
    return capt #, sum(probs) / 5


if __name__ == "__main__":
    recognize_captcha()